In [19]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.ndimage import convolve
import matplotlib.animation as animation
import json
import scipy
import os
from sklearn.mixture import GaussianMixture

import discrete_CRISPR_methods as ds
import discrete_CRISPR_sim_methods as ds2

In [20]:
def load_stuff(frame_cut):
    params = json.load(open("_params.json"))
    sim_params = json.load(open("_sim_params.json"))
    try:
        frames_f = np.load("frames_f.npy")[frame_cut:]
        frames_n = np.load("frames_n.npy")[frame_cut:]
        frames_nh = np.load("frames_nh.npy")[frame_cut:]

    except IndexError:
        frames_f = np.load("frames_f.npy")
        frames_n = np.load("frames_n.npy")
        frames_nh = np.load("frames_nh.npy")
    return params, sim_params, frames_f, frames_n, frames_nh

In [2]:
def calculate_variance(single_frame, params, sim_params):
    x_range = sim_params["xdomain"] #Initialize the spaces
    dx = sim_params["dx"]

    x_linspace = np.arange(-x_range, x_range, dx)
    coordmap = np.meshgrid(x_linspace, x_linspace)
    radius = coordmap[0]**2 + coordmap[1]**2

    rqrd = 0
    for n, r2 in zip(np.ravel(single_frame), np.ravel(radius)):
        rqrd += n*r2
    
    return rqrd/np.sum(single_frame)

In [3]:
def get_Gaussian_Fit(frames_n, params, sim_params):
    frame_means = []
    frame_cov = []
    x_range = sim_params["xdomain"] #Initialize the spaces
    dx = sim_params["dx"]

    x_linspace = np.arange(-x_range, x_range, dx)
    coordmap = np.meshgrid(x_linspace, x_linspace)

    for single_frame in frames_n:
        point_data = []
        try:
            for nonzero_ind in np.argwhere(single_frame):
                n = single_frame[nonzero_ind[0], nonzero_ind[1]]
                x_ind = coordmap[1][nonzero_ind[0], nonzero_ind[1]]
                y_ind = coordmap[0][nonzero_ind[0], nonzero_ind[1]]
                for _ in range(n):
                    point_data.append([x_ind, y_ind])

            point_data = np.array(point_data).squeeze()

            gaussian_estimator = GaussianMixture(covariance_type="full", max_iter=20)
            gaussian_estimator.fit(point_data)

            covariances = gaussian_estimator.covariances_.squeeze()
            means = gaussian_estimator.means_.squeeze()

            frame_means.append(means)
            frame_cov.append(covariances)
            
        except ValueError:
            continue

    frame_means = np.array(frame_means).squeeze()
    return frame_means, frame_cov

In [4]:
def make_ellipse(means, covariances, color = "navy"):
    v, w = np.linalg.eigh(covariances)
    u = w[0] / np.linalg.norm(w[0])
    angle = np.arctan2(u[1], u[0])
    angle = 180 * angle / np.pi  # convert to degrees
    v = 2.0 * np.sqrt(2.0) * np.sqrt(v)
    ell = mpl.patches.Ellipse(
        means, v[0], v[1], 180 + angle, color=color
    )
    return ell

In [51]:
foldername = '/media/homes/thisiszhi/code/data/ParamsSweepCross/'
subfolders = [ f.path for f in os.scandir(foldername) if f.is_dir() ]
frame_cut = 500

for current_folder in subfolders:
    os.chdir(current_folder)
    results = {
        "avg_r2":                   0,
        "avg_f":                    0,
        "avg_grp_vel":              0,
        "diff_coeff":               0,
        "avg_tt_cov":               0,
    }
    with open('_results.json', 'w') as fp:
        json.dump(results, fp)

In [52]:
for current_folder in subfolders:
    print(current_folder)
    os.chdir(current_folder)
    params, sim_params, frames_f, frames_n, frames_nh = load_stuff(frame_cut)

    frames_nvar = []
    for frame in frames_n:
        frames_nvar.append(calculate_variance(frame, params, sim_params))

    frames_nvar = np.array(frames_nvar).squeeze()
    avg_var = np.mean(frames_nvar)


    plt.figure()
    plt.title("$<r^2(t)>$ of N")
    plt.ylabel("dist")
    plt.xlabel("frame number")
    plt.plot(frames_nvar)

    try:
        x = np.arange(0,len(frames_nvar),1)
        fit = np.polyfit(x, frames_nvar, 1)
        plt.plot(x, fit[0]*x + fit[1], color="r")
    except TypeError:
        pass
    plt.savefig("variance_plot.png")
    plt.close()

    results = json.load(open("_results.json"))
    results["avg_r2"] = avg_var
    results["diff_coeff"] = fit[1]/2
    with open('_results.json', 'w') as fp:
        json.dump(results, fp)

/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#10
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#7
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#3
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#5
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#11
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#18
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#1
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#13
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in long_scalars
  del sys.path[0]


/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#12
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#16
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#17
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#9
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#15
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#8
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#2
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#6
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#4
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#14


In [53]:
for current_folder in subfolders:
    print(current_folder)
    os.chdir(current_folder)
    params, sim_params, frames_f, frames_n, frames_nh = load_stuff(frame_cut)

    try:
        frame = ma.array(frames_f[0], mask = (frames_n[0] == 0)).compressed()
        height, bins = np.histogram(frame, 25, range = (-1.1, 1.1))
        cummulative = np.zeros(height.shape)

        frame_avg = []
        for single_f, single_n in zip(frames_f, frames_n):
            frame = ma.array(single_f, mask = (single_n == 0)).compressed()
            frame_avg = np.mean(frame)
            height, _ = np.histogram(frame, 25, range = (-1.1, 1.1))
            cummulative += height

        norm_height = cummulative/np.sum(cummulative)

        plt.figure()
        plt.bar(bins[:-1], norm_height, width = 0.05)
        plt.title("Fitness Distribution")
        plt.savefig("fitness_plot.png")
        plt.close()

        results = json.load(open("_results.json"))
        results["avg_f"] = np.mean(frame_avg.squeeze())
        with open('_results.json', 'w') as fp:
            json.dump(results, fp)

    except IndexError:
        pass

/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#10
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#7
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#3
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#5
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#11
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#18
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#1
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#13
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#0


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#12
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#16
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#17
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#9
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#15
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#8
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#2
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#6
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#4
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#14


In [54]:
for current_folder in subfolders:
    print(current_folder)
    os.chdir(current_folder)
    params, sim_params, frames_f, frames_n, frames_nh = load_stuff(frame_cut)

    N = []
    for single_n in frames_n:
        n = np.sum(single_n)
        N.append(n)

    plt.figure()
    plt.plot(N)
    plt.title("Number of active phage over time")
    plt.ylabel("Population")
    plt.xlabel("Frame number")
    plt.savefig("Inflected_Population_plot.png")
    plt.close()

/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#10
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#7
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#3
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#5
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#11
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#18
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#1
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#13
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#0
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#12
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#16
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#17
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#9
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#15
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#8
/media/homes/thisiszhi/code/data/ParamsSweepCro

In [55]:
for current_folder in subfolders:
    print(current_folder)
    os.chdir(current_folder)
    params, sim_params, frames_f, frames_n, frames_nh = load_stuff(frame_cut)
    try:
        frames_means, frames_cov = get_Gaussian_Fit(frames_n, params, sim_params)

        fig, ax = plt.subplots()
        ax.plot(frames_means[:,0], frames_means[:,1])
        plt.title("Trajectory of Gaussian Fit")

        ax.scatter(frames_means[0, 0], frames_means[0, 1], color = 'red')
        ell1 = make_ellipse(frames_means[0], frames_cov[0], 'red')
        ell1.set_clip_box(ax.bbox)
        ell1.set_alpha(0.15)
        ax.add_patch(ell1)
        ax.set_aspect("equal", "datalim")

        ax.scatter(frames_means[-1, 0], frames_means[-1, 1], color = 'black')
        ell2 = make_ellipse(frames_means[-1], frames_cov[-1], 'black')
        ell2.set_clip_box(ax.bbox)
        ell2.set_alpha(0.15)
        ax.add_patch(ell2)
        ax.set_aspect("equal", "datalim")

        plt_bd = sim_params["xdomain"]
        plt.xlim(-plt_bd, plt_bd)
        plt.ylim(-plt_bd, plt_bd)

        plt.savefig("Gaussian_Fit_Plot.png")
        plt.close()
    except IndexError:
        pass

/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#10
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#7
/media/homes/thisiszhi/code/data/ParamsSweepCross/Simulation#3


In [ ]:
for current_folder in subfolders:
    print(current_folder)
    os.chdir(current_folder)
    params, sim_params, frames_f, frames_n, frames_nh = load_stuff(frame_cut)

    try:
        frames_means, frames_cov = get_Gaussian_Fit(frames_n, params, sim_params)

        frame_det_cov = []
        for cov in frames_cov:
            frame_det_cov.append(np.linalg.det(cov))
        frame_det_cov = np.array(frame_det_cov).squeeze()

        plt.figure()
        plt.title("Total Variance over Frames")
        plt.xlabel("Frame Number")
        plt.plot(frame_det_cov)
        plt.savefig("Total_Variance_Plot.png")
        plt.close()

        results = json.load(open("_results.json"))
        results["avg_tt_cov"] = np.mean(frame_det_cov)
        with open('_results.json', 'w') as fp:
            json.dump(results, fp)
    except IndexError:
        pass

/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#84
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#88
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#73
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#93
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#29
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#75
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#99
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#94
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#22
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#34
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#23
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#96
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#36
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#26
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#83
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#10
/media/homes/thisiszhi/c

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#14
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#89
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#30
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#70


In [ ]:
for current_folder in subfolders:
    print(current_folder)
    os.chdir(current_folder)
    params, sim_params, frames_f, frames_n, frames_nh = load_stuff(frame_cut)
    
    try:
        dt = sim_params["dt"]
        frames_means, frames_cov = get_Gaussian_Fit(frames_n, params, sim_params)

        frame_dist = []
        tt_time = dt
        for i in range(len(frames_means[1:])):
            diff = frames_means[i-1]-frames_means[i]
            r = diff[0]**2 + diff[1]**2
            frame_dist.append(r)
            tt_time += dt

        tt_dist = np.sum(np.array(frame_dist))

        results = json.load(open("_results.json"))
        results["avg_grp_vel"] = tt_dist/tt_time
        with open('_results.json', 'w') as fp:
            json.dump(results, fp)
    except IndexError:
        pass

/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#84
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#88
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#73
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#93
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#29
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#75
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#99
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#94
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#22
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#34
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#23
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#96
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#36
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#26
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#83
/media/homes/thisiszhi/code/data/ParamsSweepNp/Simulation#10
/media/homes/thisiszhi/c